In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import numpy as np
import warnings
from tqdm import tqdm_notebook as tqdm
warnings.filterwarnings('ignore')

In [2]:
order = pd.read_csv('../codebase/DeliveryRecord.csv')
order.head(2)

,日期,門店名稱,配送地址,下單時間,指派成功時間,騎手接單時間,到店取餐時間,取餐出發時間,訂單送達時間,預送達時間,...,Rider,EstimateArrivalGap,TripTime,TicketWaitingTime,FoodPreparationTime,TotalWaitingTime,orderHour,storeCode,lat,lon
0,2020-06-01,KFCN122,薄扶林 域多利道 216 號 趙苑三期第１座 2 座 2 樓,2020-06-01 21:56:00,2020-06-01 22:19:21,2020-06-01 22:19:29,2020-06-01 22:19:30,2020-06-01 22:19:31,2020-06-01 22:27:45,2020-06-01 22:16:00,...,Wing,-11.750000,8.233333,23.350000,23.516667,31.750000,21,KFCN122,22.282397,114.127389
1,2020-06-01,KFCN122,薄扶林 沙宣道 21 號 香港大學李嘉誠醫學院 蒙民偉樓 0 座 0 樓,2020-06-01 19:20:00,2020-06-01 19:24:49,2020-06-01 19:24:59,2020-06-01 19:25:01,2020-06-01 19:25:11,2020-06-01 19:34:58,2020-06-01 19:40:00,...,Wing,5.033333,9.783333,4.816667,5.183333,14.966667,19,KFCN122,22.282397,114.127389


In [261]:
order.columns

Index(['日期', '門店名稱', '配送地址', '下單時間', '指派成功時間', '騎手接單時間', '到店取餐時間', '取餐出發時間',
       '訂單送達時間', '預送達時間', 'Banner', 'Instant Order', '收貨地址坐標', 'Rider',
       'EstimateArrivalGap', 'TripTime', 'TicketWaitingTime',
       'FoodPreparationTime', 'TotalWaitingTime', 'orderHour', 'storeCode',
       'lat', 'lon'],
      dtype='object')

In [262]:
order['orderTime'] = pd.to_datetime(order['下單時間'])
order['orderMonth'] = pd.to_datetime(order['下單時間']).dt.month
order['departureTime'] = pd.to_datetime(order['取餐出發時間'])
order['Origin'] = order.apply(lambda x: Point(float(x['lon']),float(x['lat'])), axis=1)
order['Destination'] = order['收貨地址坐標'].apply(lambda x: Point(float(x.split(',')[1]),float(x.split(',')[0])))
order['extraTime'] = (pd.to_datetime(order['預送達時間']) - pd.to_datetime(order['訂單送達時間'])) / np.timedelta64(1, 'm')
order = order.sort_values(by=['orderTime'])

In [263]:
# extra time has to be larger than 0 otherwise cannot be combined
order = order.loc[(order['extraTime'] > 0)&(order['orderMonth']==6)]

In [264]:
order = order[['departureTime','extraTime','TripTime','Origin','Destination']]
order['CombineWith'] = 0
order['path'] = 0
order.head()

,departureTime,extraTime,TripTime,Origin,Destination,CombineWith,path
9950,2020-06-01 11:07:50,6.583333,24.733333,POINT (114.2023552 22.33722430000001),POINT (114.2192197311637 22.32888477511504),0,0
54434,2020-06-01 11:21:19,18.633333,6.683333,POINT (114.195675 22.3968516),POINT (114.1981143714086 22.39675192356175),0,0
85834,2020-06-01 11:24:12,24.933333,7.550000,POINT (113.9984839 22.4561072),POINT (114.0007872670482 22.45439350218214),0,0
76492,2020-06-01 11:23:37,24.750000,10.366667,POINT (114.1397926 22.284362),POINT (114.1373175990397 22.28297236226782),0,0
34910,2020-06-01 11:21:25,23.500000,8.983333,POINT (114.1885344 22.2787415),POINT (114.1990351523666 22.28567181299781),0,0


In [265]:
# set crs for both origin and destination
order = gpd.GeoDataFrame(order, geometry='Origin')
order = order.set_crs(epsg=4326)
order = order.to_crs(epsg=2263)
order = gpd.GeoDataFrame(order, geometry='Destination')
order = order.set_crs(epsg=4326)
order = order.to_crs(epsg=2263)

In [266]:
order.index = range(len(order))

In [267]:
order.head()

,departureTime,extraTime,TripTime,Origin,Destination,CombineWith,path
0,2020-06-01 11:07:50,6.583333,24.733333,POINT (-27758116.504 36312344.939),POINT (-27763335.015 36308021.133),0,0
1,2020-06-01 11:21:19,18.633333,6.683333,POINT (-27736187.329 36305881.206),POINT (-27736551.692 36305095.873),0,0
2,2020-06-01 11:24:12,24.933333,7.550000,POINT (-27688630.414 36361858.000),POINT (-27689546.248 36361354.040),0,0
3,2020-06-01 11:23:37,24.750000,10.366667,POINT (-27768302.562 36340557.578),POINT (-27768457.646 36341572.090),0,0
4,2020-06-01 11:21:25,23.500000,8.983333,POINT (-27776878.634 36325373.329),POINT (-27775853.006 36320919.142),0,0


In [268]:
for i in tqdm(range(len(order))):
# for i in range(10):
    if order.iloc[i]['CombineWith'] == 0:
        extraTime = order.iloc[i]['extraTime']
        departureTime = order.iloc[i]['departureTime']
        Origin = order.iloc[i]['Origin']
        Destination = order.iloc[i]['Destination']
        TripTime = order.iloc[i]['TripTime']
        TripLength = LineString([Origin,Destination]).length
        speed = TripLength / TripTime
        bufferDist = speed * extraTime
        TotalTime1 = TripTime + extraTime
        
'''        dataset is sorted by order time, so ridesharing candidates have to be orders later than 
        the current selected one'''

        candidate = order.iloc[i+1:]
        candidate = candidate.loc[candidate['CombineWith'] == 0]
        '''order ready to pick time should be earlier than order 1 departure time + order 1 extra time, 
        otherwise we will waste all the extra time on waiting for the second order'''
        candidate = candidate.loc[((candidate['departureTime'] - departureTime) / np.timedelta64(1, 'm')) < extraTime]
        '''trip duration between two origins and two destinations cannot be longer than order 1 extra time,
        use buffer zone to filter'''
        candidate['OriginIn'] = candidate['Origin'].apply(lambda x: Origin.buffer(bufferDist).intersects(x))
        candidate['DestinationIn'] = candidate['Destination'].apply(lambda x: Destination.buffer(bufferDist).intersects(x))
        candidate = candidate.loc[(candidate['OriginIn']==True)&(candidate['DestinationIn']==True)]
        
        '''4 potention path: O1-O2-D1-D2, O1-O2-D2-D1, O2-O1-D1-D2, O2-O1-D2-D1, calculate arrival time for each order,
        if both order could be delivered before estimated time, stop calculation'''
        if len(candidate) > 0:
            for ind in candidate.index.tolist():
                combined = 0
                candidateSelect = candidate.loc[candidate.index==ind]
                Origin2 = candidateSelect.iloc[0]['Origin']
                Destination2 = candidateSelect.iloc[0]['Destination']
                TripTime2 = candidateSelect.iloc[0]['TripTime']
                extraTime2 = candidateSelect.iloc[0]['extraTime']
                TotalTime2 = TripTime2 + extraTime2
       
                path1 = LineString([Origin,Origin2,Destination])
                Order1Time = path1.length / speed
                path2 = LineString([Origin,Origin2,Destination,Destination2])
                Order2Time = path2.length / speed
                
                if Order1Time < TotalTime1 and Order2Time < TotalTime2:
                    order.loc[order.index==ind,'CombineWith'] = i
                    order.loc[order.index==i,'CombineWith'] = ind
                    
                    order.loc[order.index==i,'path'] = 'Origin1,Origin2,Destination1,Destination2'
                    order.loc[order.index==ind,'path'] = 'Origin1,Origin2,Destination1,Destination2'
                    
                    combined = 1
                else:
                    path1 = LineString([Origin,Origin2,Destination2,Destination])
                    Order1Time = path1.length / speed
                    path2 = LineString([Origin,Origin2,Destination2])
                    Order2Time = path1.length / speed
                    if Order1Time < TotalTime1 and Order2Time < TotalTime2:
                        order.loc[order.index==ind,'CombineWith'] = i
                        order.loc[order.index==i,'CombineWith'] = ind
                        order.loc[order.index==i,'path'] = 'Origin1,Origin2,Destination2,Destination1'
                        order.loc[order.index==ind,'path'] = 'Origin1,Origin2,Destination2,Destination1'
                        
                        combined = 1
                    else:
                        path1 = LineString([Origin2,Origin,Destination])
                        Order1Time = path1.length / speed
                        path2 = LineString([Origin2,Origin,Destination,Destination2])
                        Order2Time = path1.length / speed
                        if Order1Time < TotalTime1 and Order2Time < TotalTime2:
                            order.loc[order.index==ind,'CombineWith'] = i
                            order.loc[order.index==i,'CombineWith'] = ind
                            order.loc[order.index==i,'path'] = 'Origin2,Origin1,Destination1,Destination2'
                            order.loc[order.index==ind,'path'] = 'Origin2,Origin1,Destination1,Destination2'
                            
                            combined = 1
                        else:
                            path1 = LineString([Origin2,Origin,Destination2,Destination])
                            Order1Time = path1.length / speed
                            path2 = LineString([Origin2,Origin,Destination2])
                            Order2Time = path1.length / speed
                        if Order1Time < TotalTime1 and Order2Time < TotalTime2:
                            order.loc[order.index==ind,'CombineWith'] = i
                            order.loc[order.index==i,'CombineWith'] = ind
                            order.loc[order.index==i,'path'] = 'Origin2,Origin1,Destination2,Destination1'
                            order.loc[order.index==ind,'path'] = 'Origin2,Origin1,Destination2,Destination1'
                            
                            combined = 1
                if combined > 0:
                    break
                            
                            

In [257]:
order.head(10)

,departureTime,extraTime,TripTime,Origin,Destination,CombineWith,path
0,2020-06-01 11:07:50,6.583333,24.733333,POINT (-27758116.504 36312344.939),POINT (-27763335.015 36308021.133),0,0
1,2020-06-01 11:21:19,18.633333,6.683333,POINT (-27736187.329 36305881.206),POINT (-27736551.692 36305095.873),0,0
2,2020-06-01 11:24:12,24.933333,7.550000,POINT (-27688630.414 36361858.000),POINT (-27689546.248 36361354.040),11,"Origin1,Origin2,Destination1,Destination2"
3,2020-06-01 11:23:37,24.750000,10.366667,POINT (-27768302.562 36340557.578),POINT (-27768457.646 36341572.090),0,0
4,2020-06-01 11:21:25,23.500000,8.983333,POINT (-27776878.634 36325373.329),POINT (-27775853.006 36320919.142),7,"Origin1,Origin2,Destination2,Destination1"
5,2020-06-01 11:18:49,10.316667,19.666667,POINT (-27779955.955 36313752.109),POINT (-27779282.761 36314551.879),0,0
6,2020-06-01 11:39:10,17.416667,4.350000,POINT (-27739224.959 36329578.557),POINT (-27741806.638 36327472.888),16,"Origin1,Origin2,Destination1,Destination2"
7,2020-06-01 11:42:17,13.150000,15.033333,POINT (-27780490.206 36327570.990),POINT (-27776997.710 36327106.772),4,"Origin1,Origin2,Destination2,Destination1"
8,2020-06-01 11:41:32,16.316667,6.550000,POINT (-27736187.329 36305881.206),POINT (-27736116.658 36305535.003),0,0
9,2020-06-01 11:44:54,13.550000,9.583333,POINT (-27736187.329 36305881.206),POINT (-27730715.272 36298381.601),0,0


In [270]:
order.loc[order['CombineWith'] > 0]

,departureTime,extraTime,TripTime,Origin,Destination,CombineWith,path
2,2020-06-01 11:24:12,24.933333,7.550000,POINT (-27688630.414 36361858.000),POINT (-27689546.248 36361354.040),11,"Origin1,Origin2,Destination1,Destination2"
4,2020-06-01 11:21:25,23.500000,8.983333,POINT (-27776878.634 36325373.329),POINT (-27775853.006 36320919.142),7,"Origin1,Origin2,Destination2,Destination1"
6,2020-06-01 11:39:10,17.416667,4.350000,POINT (-27739224.959 36329578.557),POINT (-27741806.638 36327472.888),16,"Origin1,Origin2,Destination1,Destination2"
7,2020-06-01 11:42:17,13.150000,15.033333,POINT (-27780490.206 36327570.990),POINT (-27776997.710 36327106.772),4,"Origin1,Origin2,Destination2,Destination1"
11,2020-06-01 11:48:29,11.083333,18.083333,POINT (-27688630.414 36361858.000),POINT (-27688650.968 36359426.053),2,"Origin1,Origin2,Destination1,Destination2"
...,...,...,...,...,...,...,...
20444,2020-06-30 21:54:24,21.616667,21.250000,POINT (-27748718.204 36332798.654),POINT (-27744487.986 36331854.239),20440,"Origin2,Origin1,Destination2,Destination1"
20445,2020-06-30 22:01:29,23.216667,4.766667,POINT (-27739224.959 36329578.557),POINT (-27739688.180 36326744.950),20453,"Origin2,Origin1,Destination2,Destination1"
20453,2020-06-30 22:14:05,22.650000,12.283333,POINT (-27733670.129 36337249.377),POINT (-27735244.729 36333922.149),20445,"Origin2,Origin1,Destination2,Destination1"
20454,2020-06-30 22:39:16,28.500000,5.500000,POINT (-27784532.285 36308784.776),POINT (-27792596.404 36305453.078),20456,"Origin2,Origin1,Destination2,Destination1"
